# Data Cleaning Process 
1. Check and remove duplicates
2. Audit and handle missing values
3. Handle data types
4. Rename the columns
5. Outlier detection
6. Data validation
7. Feature engineering
8. Standardize categorical data
9. Data quality check 

In [1]:
import pandas as pd
import numpy as np 

In [2]:
df = pd.read_csv("avocado.csv")

In [3]:
df.head()
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18249 entries, 0 to 18248
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    18249 non-null  int64  
 1   Date          18249 non-null  object 
 2   AveragePrice  18249 non-null  float64
 3   Total_Volume  18249 non-null  float64
 4   4046          18249 non-null  float64
 5   4225          18249 non-null  float64
 6   4770          18249 non-null  float64
 7   Total_Bags    18249 non-null  float64
 8   Small_Bags    18249 non-null  float64
 9   Large_Bags    18249 non-null  float64
 10  XLarge_Bags   18249 non-null  float64
 11  type          18249 non-null  object 
 12  year          18249 non-null  int64  
 13  region        18249 non-null  object 
dtypes: float64(9), int64(2), object(3)
memory usage: 1.9+ MB


(18249, 14)

##### Total Rows (18249)
##### Total Columns (14)

In [4]:
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
)

In [5]:
df = df.drop(columns = ["unnamed:_0"], errors = "ignore")

##### Removed non-informative index column to reduce dataset noise

In [6]:
df['date'] = pd.to_datetime(df['date'])

In [7]:
numeric_cols = [
    'averageprice', 'total_volume', '4046', '4225', '4770', 
    'total_bags', 'small_bags','large_bags', 'xlarge_bags'
]
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors = 'coerce')

In [8]:
df.isnull().sum()

date            0
averageprice    0
total_volume    0
4046            0
4225            0
4770            0
total_bags      0
small_bags      0
large_bags      0
xlarge_bags     0
type            0
year            0
region          0
dtype: int64

##### Prices cannot be null
##### Volume can be 0
##### Sum must match total_bags

In [9]:
df = df.dropna(subset = ['averageprice'])

In [10]:
df['bags_check'] = (
    df['small_bags'] +
    df['large_bags'] +
    df['xlarge_bags']
)
df['bags_match'] = df['bags_check'] == df['total_bags']

In [11]:
df[df['bags_match'] == False].head()

,date,averageprice,total_volume,4046,4225,4770,total_bags,small_bags,large_bags,xlarge_bags,type,year,region,bags_check,bags_match
6,2015-11-15,0.99,83453.76,1368.92,73672.72,93.26,8318.86,8196.81,122.05,0.00,conventional,2015,Albany,8318.86,False
16,2015-09-06,1.11,99172.96,879.45,90062.62,240.79,7990.10,7762.87,227.23,0.00,conventional,2015,Albany,7990.10,False
19,2015-08-16,1.33,80043.78,539.65,68666.01,394.90,10443.22,10297.68,145.54,0.00,conventional,2015,Albany,10443.22,False
24,2015-07-12,1.05,124055.31,672.25,94693.52,4257.64,24431.90,24290.08,108.49,33.33,conventional,2015,Albany,24431.90,False
26,2015-06-28,1.37,89534.81,664.23,57545.79,4662.71,26662.08,26311.76,350.32,0.00,conventional,2015,Albany,26662.08,False


In [12]:
df.duplicated().sum()

np.int64(0)

##### No duplicates were found in this data 

In [13]:
q1 = df['averageprice'].quantile(0.25)
q3 = df['averageprice'].quantile(0.75)
iqr = q3 - q1

df = df[
    (df['averageprice'] >= q1 - 1.5 * iqr) &
    (df['averageprice'] <= q3 + 1.5 * iqr)
]

In [14]:
df.loc[:, 'year'] = df['date'].dt.year
df.loc[:, 'month'] = df['date'].dt.month

In [15]:
df.loc[:, 'price_per_unit'] = df['total_volume'] / df['averageprice']

In [16]:
df.loc[:, 'type'] = df['type'].str.lower()
df.loc[:, 'region'] = df['region'].str.title()

In [17]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 18040 entries, 0 to 18248
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            18040 non-null  datetime64[ns]
 1   averageprice    18040 non-null  float64       
 2   total_volume    18040 non-null  float64       
 3   4046            18040 non-null  float64       
 4   4225            18040 non-null  float64       
 5   4770            18040 non-null  float64       
 6   total_bags      18040 non-null  float64       
 7   small_bags      18040 non-null  float64       
 8   large_bags      18040 non-null  float64       
 9   xlarge_bags     18040 non-null  float64       
 10  type            18040 non-null  object        
 11  year            18040 non-null  int64         
 12  region          18040 non-null  object        
 13  bags_check      18040 non-null  float64       
 14  bags_match      18040 non-null  bool          
 15  month  

,date,averageprice,total_volume,4046,4225,4770,total_bags,small_bags,large_bags,xlarge_bags,year,bags_check,month,price_per_unit
count,18040,18040.000000,1.804000e+04,1.804000e+04,1.804000e+04,1.804000e+04,1.804000e+04,1.804000e+04,1.804000e+04,18040.000000,18040.000000,1.804000e+04,18040.000000,1.804000e+04
mean,2016-08-11 03:44:51.618625536,1.390967,8.603374e+05,2.963771e+05,2.985006e+05,2.310400e+04,2.423535e+05,1.842700e+05,5.494114e+04,3142.384961,2016.141796,2.423535e+05,6.157650,8.266546e+05
min,2015-01-04 00:00:00,0.440000,8.456000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,2015.000000,0.000000e+00,1.000000,5.318239e+01
25%,2015-10-18 00:00:00,1.090000,1.101439e+04,8.881150e+02,3.017740e+03,0.000000e+00,5.323757e+03,3.006912e+03,1.376900e+02,0.000000,2015.000000,5.323758e+03,3.000000,7.075483e+03
50%,2016-08-07 00:00:00,1.360000,1.118385e+05,9.114650e+03,3.012849e+04,1.987500e+02,4.137342e+04,2.730882e+04,2.747005e+03,0.000000,2016.000000,4.137342e+04,6.000000,8.397683e+04
75%,2017-06-04 00:00:00,1.650000,4.399874e+05,1.134827e+05,1.536981e+05,6.401722e+03,1.125642e+05,8.458404e+04,2.258917e+04,141.820000,2017.000000,1.125642e+05,9.000000,3.714764e+05
max,2018-03-25 00:00:00,2.490000,6.250565e+07,2.274362e+07,2.047057e+07,2.546439e+06,1.937313e+07,1.338459e+07,5.719097e+06,551693.650000,2018.000000,1.937313e+07,12.000000,7.926553e+07
std,NaN,0.379588,3.472312e+06,1.271907e+06,1.210672e+06,1.080566e+05,9.916143e+05,7.502378e+05,2.453098e+05,17791.921748,0.941474,9.916143e+05,3.538889,3.425397e+06


##### Clean Schema
##### BI ready
##### Optimized for Tableau

In [18]:
df.to_csv("avocado_cleaned.csv", index=False)